<a href="https://colab.research.google.com/github/SergeyHSE/ALSalgorithm.github.io/blob/main/ALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work, we will find similar movies and users using the ALS algorithm,
implement the calculation of the NDCG metric, and investigate
the effect of the dimensionality of hidden representations on the performance of the algorithm.

Dataset = MovieLens

In [1]:
import zipfile
from collections import defaultdict, Counter
import datetime
from scipy import linalg
import scipy.sparse as sps
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2023-12-02 14:54:32--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  19.5MB/s    in 0.3s    

2023-12-02 14:54:33 (19.5 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [ ]:
with zipfile.ZipFile("ml-1m.zip", 'r') as z:
    file_content = z.read('ml-1m/README')
    decoded_content = file_content.decode('iso-8859-1')  # Decode the content assuming it's in UTF-8

    # Replace single newlines with a special token
    content_with_token = decoded_content.replace('\n', '||NEWLINE||')

    # Replace double newlines with spaces
    formatted_content = ' '.join(para.strip() for para in content_with_token.split('\n\n'))

    # Restore single newlines
    formatted_content = formatted_content.replace('||NEWLINE||', '\n')

    print(formatted_content)

SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:

     * The user may not state or imply any endorsement from the
       University of Minnesota or the GroupLens Research Group.

     * The user must acknowledge the use of the data set in
       publications resulting from the use of the data set
       (see below for citation information).

     * The user may not redistribute the data without separate
       permission.

     * The user may not use this information for any commercial or
       revenue-bearing purposes without first obtaining permission
       from a facult

In [3]:
#Let's unpack the data and see how it's organized.

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    print("files in archive")
    print(z.namelist())
    print("movies")
    with z.open("ml-1m/movies.dat") as m:
        print(str(m.readline()).split("::"))
    print("users")
    with z.open("ml-1m/users.dat") as m:
        print(str(m.readline()).split("::"))
    print("ratings")
    with z.open("ml-1m/ratings.dat") as m:
        print(str(m.readline()).split("::"))

files in archive
['ml-1m/', 'ml-1m/movies.dat', 'ml-1m/ratings.dat', 'ml-1m/README', 'ml-1m/users.dat']
movies
['b"1', 'Toy Story (1995)', 'Animation|Children\'s|Comedy\\n"']
users
["b'1", 'F', '1', '10', "48067\\n'"]
ratings
["b'1", '1193', '5', "978300760\\n'"]


We can see that the archive contains information about movies.
This is the movieId of the movie, title and genre.
About users we know userId, gender (F, M), age, coded employment information and zip-code.
And the rating information:
userId, movieId, rating and the moment in time when the rating was made.
Let's read the data.

In [4]:
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.decode('iso-8859-1').strip().split("::")
            MovieID = int(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}

    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = list(zip(fields, line.decode('iso-8859-1').strip().split("::")))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[int(row[0][1])] = data

    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for line in m:
            UserID, MovieID, Rating, Timestamp = line.decode('iso-8859-1').strip().split("::")
            UserID = int(UserID)
            MovieID = int(MovieID)
            Rating = int(Rating)
            Timestamp = int(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

In [5]:
len(users), len(ratings), len(movies)

(6040, 6040, 3883)

In [6]:
times = []
for user_ratings in ratings.values():
    times.extend([x[2] for x in user_ratings])
times = sorted(times)

threshold_time = times[int(0.8 * len(times))]
threshold_time

datetime.datetime(2000, 12, 2, 14, 52, 18)

In [7]:
train = []
test = []

for user_id, user_ratings in ratings.items():
    train.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] <= threshold_time)
    test.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] > threshold_time)
print("ratings in train:", len(train))
print("ratings in test:", len(test))

ratings in train: 800168
ratings in test: 200041


In [8]:
train_by_user = defaultdict(list)
test_by_user = defaultdict(list)
for u, i, r in train:
    train_by_user[u].append((i, r))
for u, i, r in test:
    test_by_user[u].append((i, r))

train_by_item = defaultdict(list)
for u, i, r in train:
    train_by_item[i].append((u, r))

In [9]:
#Let's count the number of users and movies

n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1
n_users, n_items

(6041, 3953)

In [10]:
np.random.seed(0)
LATENT_SIZE = 10
N_ITER = 20

# regularizers
lambda_p = 0.2
lambda_q = 0.001

# latent representations
p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))


**ALS**

In this approach, the score $r_{ui}$ of user $u$ given to movie $i$,

is sought as the scalar product of vectors $p_u$ and $q_i$ in some space $\mathbb{R}^K$ of latent features:

$$
	\hat{r}_{ui} = p_u^T q_i
$$


In other words, the model finds a feature space in which we describe both movies and users and in which the rating is a measure of the closeness between movies and users.

To tune the model, we will minimize the following error:
	$$
	\sum_{(u, i, r_{ui})} (r_{ui} - p_u^T q_i)^2 + \lambda_{p} p_u^T p_u + \lambda_{q} q_i^T q_i,
	$$
	where the summation is over all triples $(u, i, r_{ui})$ of the sample, the summands with $\lambda_{p}$ and $\lambda_{q}$ are added for regularization.

Now let us make a matrix $P$ from vectors $p_u$ and a matrix $Q$ from vectors $q_i$. By matrix $Q[u] \in \mathbb{R}^{n_u \times K}$ we denote the submatrix of matrix $Q$ only for goods evaluated by user $u$, where $n_u$ is the number of evaluations of user $u$.

The reconfiguration step $p_u$ for a fixed matrix $Q$ reduces to the ridge regression setup and looks like this:
	$$
	A_u = Q[u]^T Q[u] \\\\
	d_u = Q[u]^T r_u \\\
	p_u = (\lambda_p n_u I + A_u)^{-1} d_u
	$$


In [11]:
def compute_p(p, q, train_by_user):
    for u, rated in train_by_user.items():
        rated_items = [i for i, _ in rated]
        rated_scores = np.array([r for _, r in rated])
        Q = q[rated_items, :]
        A = (Q.T).dot(Q)
        d = (Q.T).dot(rated_scores)
        p[u, :] = np.linalg.solve(lambda_p * len(rated_items) * np.eye(LATENT_SIZE) + A, d)
    return p

In [12]:
def compute_q(p, q, train_by_item):
    for i, rated in train_by_item.items():
        rated_users = [j for j, _ in rated]
        rated_scores = np.array([s for _, s in rated])
        P = p[rated_users, :]
        A = (P.T).dot(P)
        d = (P.T).dot(rated_scores)
        q[i, :] = np.linalg.solve(lambda_q * len(rated_users) * np.eye(LATENT_SIZE) + A, d)
    return q

In [13]:
def train_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])

In [14]:
for iter in range(N_ITER):
    p = compute_p(p, q, train_by_user)
    q = compute_q(p, q, train_by_item)

    predictions = p.dot(q.T)

    print(iter, train_error_mse(predictions), test_error_mse(predictions))

0 0.034254066990950016 0.16161048497212951
1 0.030645740984182004 0.15155084906221655
2 0.027045334327151088 0.14384734040494065
3 0.025813288873051232 0.1369731449899051
4 0.025347613143060384 0.13077566964080364
5 0.02509638013540347 0.12524794035311057
6 0.02493404752684068 0.12031008916560125
7 0.02482027996454204 0.11587970123247371
8 0.02473748090535384 0.11188957847429643
9 0.024677350034760324 0.1082859231790354
10 0.024634483994446333 0.10502502426863132
11 0.024604361404763415 0.10207014908552949
12 0.024583346331205864 0.09938950190571325
13 0.024568755099793158 0.09695506282023533
14 0.024558698531058874 0.09474199207447921
15 0.024551877533063843 0.09272824318660171
16 0.02454739123798561 0.09089423607528814
17 0.024544605124752126 0.08922255977615293
18 0.02454306682449277 0.08769769701279093
19 0.024542448316282706 0.08630578168734016


**Calculate third movies similared 'Star WAr (1980)**

In [15]:
# Search for the movie title in the dictionary
search_title = 'Star Wars: Episode V - The Empire Strikes Back (1980)'

found_movie = None
for movie_id, movie_info in movies.items():
    if movie_info['Title'] == search_title:
        found_movie = {movie_id: movie_info}

print(found_movie)


{1196: {'Title': 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Genres': ['Action', 'Adventure', 'Drama', 'Sci-Fi', 'War']}}


In [16]:
p.shape, q.shape

((6041, 10), (3953, 10))

In [17]:
#Let's calculate the scalar product of its embedding with the rest of the movie

from sklearn.metrics.pairwise import cosine_similarity

movie_cos_simiraity = cosine_similarity(q)
movie_cos_simiraity.shape

(3953, 3953)

In [18]:
movie_star_war = movie_cos_simiraity[1196]
movie_star_sorted = np.argsort(movie_star_war)[::-1]

first_movies = 10
print(f'Previous {first_movies} movies least similar to Movie {movie_star_war}:')
cosine = []
for i in range(1, first_movies + 1):
    similar_movie_index = movie_star_sorted[i]
    similarity = movie_star_war[similar_movie_index]
    print(f'Movie {similar_movie_index}: Similarity = {similarity:.4f}')
    cosine.append((similarity))

answer1 = sum(cosine[:3])
print(answer1)

Previous 10 movies least similar to Movie [0.66139005 0.90589833 0.81730857 ... 0.64824627 0.64386052 0.81881035]:
Movie 260: Similarity = 0.9967
Movie 1210: Similarity = 0.9808
Movie 1198: Similarity = 0.9768
Movie 1291: Similarity = 0.9709
Movie 2716: Similarity = 0.9661
Movie 1374: Similarity = 0.9646
Movie 3187: Similarity = 0.9597
Movie 592: Similarity = 0.9579
Movie 1242: Similarity = 0.9466
Movie 1197: Similarity = 0.9464
2.9543195369287503


In [19]:
#found names all similar movies

found_mov = None
for movie_id, movie_info in movies.items():
    if movie_id == 260:
        found_mov = {movie_id: movie_info}
        print(found_mov)
    elif movie_id == 1210:
        found_mov = {movie_id: movie_info}
        print(found_mov)
    elif movie_id == 1198:
        found_mov = {movie_id: movie_info}
        print(found_mov)
    elif movie_id == 1196:
        found_mov = {movie_id: movie_info}
        print(found_mov)

{260: {'Title': 'Star Wars: Episode IV - A New Hope (1977)', 'Genres': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi']}}
{1196: {'Title': 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Genres': ['Action', 'Adventure', 'Drama', 'Sci-Fi', 'War']}}
{1198: {'Title': 'Raiders of the Lost Ark (1981)', 'Genres': ['Action', 'Adventure']}}
{1210: {'Title': 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Genres': ['Action', 'Adventure', 'Romance', 'Sci-Fi', 'War']}}
